In [55]:
import json, os, re
from collections import defaultdict

In [56]:
# 1000 normostran
HAHAKOEFICIENT = 1800 * 1000

In [57]:
def to_utc(date):
    def trnsl(month):
        months = ["ledna", "února", "března", "dubna", "května", "června", "července", "srpna", "září", "října", "listopadu", "prosince"]
        return months.index(month) + 1
    date_parts = date.split()
    return date_parts[2] + "-" \
        + str(trnsl(date_parts[1])).zfill(2) + "-" \
        + date_parts[0].strip(".").zfill(2)

In [ ]:
# date/wc
wcdict = {}
filelist = sorted(root+"\\"+filename for root, dirs, files in list(os.walk('json'))[1:] for filename in files)
for filename in filelist:
    wc = 0
    date = ""
    with open(filename, encoding="utf8") as file:
        file = json.load(file)
        for el in file:
            if el:
                if not date:
                    date = el["date"]
                wc += len(el["speech"])
                
    if date not in wcdict: wcdict[date] = []
    wcdict[date].append(wc)

In [ ]:
with open("data.json", encoding="utf8") as file:
    smich_data = json.load(file)

for smich in smich_data:
    smich_count = len(re.findall("\(.*?[sS]mích.*?\)", smich["speech"]))
    smich["haha"] = smich_count
    smich["wc"] = sum(wcdict[smich["date"]])
    smich["date"] = to_utc(smich["date"])
    del smich["speaker"]
    del smich["link"]
    del smich["speech"]
smich_data

In [118]:
# merge
data = {date: {"haha": 0, "wc": 0} for date in set([el["date"] for el in smich_data])}
for smich in smich_data:
    data[smich["date"]]["haha"] += smich["haha"]
    data[smich["date"]]["wc"] = smich["wc"]

In [119]:
# normalize
for date in data:
    data[date]["hahaIndex"] = round(data[date]["haha"] / data[date]["wc"] * HAHAKOEFICIENT)

In [120]:
with open("results_timeline.json","w",encoding="utf8") as file:
    json.dump(data, file, ensure_ascii=False)